# MR ASH GTEx genes
Here we analyze a few GTEx genes with `mr-ash`.

In [ ]:
%cd ~/Documents/GTEx/MRASH/TOY

In [ ]:
[global]
depends: Py_Module('h5py')
import h5py
parameter: genotype_file = 'TY.genotype.h5'
parameter: expr_file = 'TY.expr.h5'
parameter: covar_file = 'TY.covariates.h5'
parameter: tissues = list(h5py.File(expr_file, libver='latest').keys())
f = h5py.File(genotype_file, libver='latest')
parameter: genes = []
for k in f:
    genes.extend(['{}/{}'.format(k, x) for x in f[k]])

In [ ]:
[1]
depends: R_library('rhdf5'), R_library('tools')
input: for_each = ['genes', 'tissues']
output: "${_tissues}_${_genes!b}.rds"
task:
R:
load_data = function(genotype_file, expr_file,cov_file, geno_table, expr_table,cov_table) {
  geno = h5read(genotype_file, geno_table)
  gdata = geno$block0_values
  colnames(gdata) = geno$axis1
  rownames(gdata) = geno$axis0
  
  expr = h5read(expr_file, expr_table)
  edata = expr$block0_values
  # colnames(edata) = expr$axis1
  colnames(edata) = tools::file_path_sans_ext(expr$axis1)
  # rownames(edata) = expr$axis0
  rownames(edata) = apply(sapply(strsplit(expr$axis0,"-"), `[`, c(1,2)), 2, function(x) paste(x, collapse = '-'))
  
  index = which(duplicated(row.names(edata)))
  edata = edata[-index,]
  # edata = data.frame(edata)
  covariate <- h5read(cov_file, cov_table)
  cdata = covariate$block0_values
  colnames(cdata) = apply(sapply(strsplit(covariate$axis1,"-"), `[`, c(1,2)), 2, function(x) paste(x, collapse = '-'))
  # rownames(edata) = expr$axis0
  rownames(cdata) = covariate$axis0 
  cdata = t(cdata)[-index,]
  gdata = data.frame(gdata)
  # I want to use merge but later
  # index_overlap = which(row.names(gdata) %in% row.names(edata))
  edata = edata[, basename(geno_table)]
  edata = data.frame(edata)
  edata$ID = rownames(edata)
  gdata$ID = rownames(gdata)
  output = merge(x = edata, y = gdata, by = "ID", all.x = TRUE)
  # gdata = gdata[index_overlap,]
  return(list(X=as.matrix(output[,-c(1,2)]), y = as.vector(output$edata), Z = as.matrix(cdata)))
}

autoselect.mixsd = function(betahat,sebetahat,mult = sqrt(2)){
  sebetahat=sebetahat[sebetahat!=0] #To avoid exact measure causing (usually by mistake)
  sigmaamin = min(sebetahat)/10 #so that the minimum is small compared with measurement precision
  if(all(betahat^2<=sebetahat^2)){
    sigmaamax = 8*sigmaamin #to deal with the occassional odd case where this could happen; 8 is arbitrary
  }else{
    sigmaamax = 2*sqrt(max(betahat^2-sebetahat^2)) #this computes a rough largest value you'd want to use, based on idea that sigmaamax^2 + sebetahat^2 should be at least betahat^2   
  }
  if(mult==0){
    return(c(0,sigmaamax/2))
  }else{
    npoint = ceiling(log2(sigmaamax/sigmaamin)/log2(mult))
    return(mult^((-npoint):0) * sigmaamax)
  }
}

initial_step = function(X,y,Z = NULL){
  P = dim(X)[2]
  output = matrix(0,nrow = P,ncol = 2)
  for(i in 1:P){
    if(is.null(Z)){
      g = summary(lm(y~X[,i]))
    } else{
      g = summary(lm(y~X[,i]+Z))
    }
    
    output[i,] = g$coefficients[2,1:2]
  }
  return(list(betahat = output[,1],sebetahat = output[,2]))
}
                                                                  
analyze = function(genename = '/chr4/ENSG00000145214', tissue = '/Lung', out = 'test.rds'){
  library(rhdf5)
  genotype_file = ${genotype_file!r}
  expr_file = ${expr_file!r}
  geno_table = genename
  expr_table = tissue 
  gene = basename(geno_table)
  cov_file = ${covar_file!r}
  cov_table = expr_table
  dat = load_data(genotype_file = genotype_file,
                  expr_file = expr_file,
                  cov_file = cov_file,
                  geno_table = geno_table,
                  expr_table = expr_table,
                  cov_table = cov_table)
  X = as.matrix(dat$X)
  X = X[,which(colSums(X)!=0)]
  if ((nrow(X) == 0) || (ncol(X) == 0)) {
  	saveRDS(list(), out)
  } else {
  storage.mode(X) <- "double"
  y = as.vector(dat$y)
  Z = as.matrix(dat$Z)
  initial = initial_step(X,y,Z)
  mixsd = autoselect.mixsd(initial$betahat,initial$sebetahat)
  logdata = capture.output({res = varbvs::varbvsmix(X, Z, y, sa = c(0,mixsd^2)) })
  mrash_out = list(betahat = rowSums(res$alpha * res$mu))
  ash_out = ashr::ash(initial$betahat,initial$sebetahat,mixcompdist = "normal")
  saveRDS(list(ash = ash_out, uni = initial, mr_ash = mrash_out, logdata = logdata), out)
}
}
analyze(genename = "/${_genes}", tissue = "/${_tissues}", out = ${_output!r})

In [ ]:
%sosrun -J 40